<a href="https://colab.research.google.com/github/MaitreyaM/Pytorch/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q transformers accelerate datasets bitsandbytes peft trl huggingface-hub safetensors pandas matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from huggingface_hub import notebook_login

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
from transformers import AutoModelForCausalLM , AutoTokenizer
from transformers import pipeline
import torch

In [14]:
model_id="meta-llama/Llama-3.2-1B-Instruct"
device="auto"


tokenizer=AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.bfloat16,device_map=device)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [15]:
tokenizer.pad_token= tokenizer.eos_token


In [16]:
generation_pipeline = pipeline(task="text-generation", model=model,tokenizer=tokenizer)

generation_pipeline("Hello how are you",max_new_tokens=25)

Device set to use cuda:0


[{'generated_text': "Hello how are you today? I'm excited to see you today! I'm looking forward to our conversation today. How can I help you with"}]

In [17]:
##BATCH GENERATION

In [18]:
generation_pipeline([
    "Hello how are you",
    "What is Agentic AI"
],max_new_tokens=25)

[[{'generated_text': 'Hello how are you?\nI am just browsing the web and I came across a website that was using the default settings of the browser. I think'}],
 [{'generated_text': 'What is Agentic AI?\nAgentic AI refers to a new generation of artificial intelligence that is designed to be more autonomous, self-aware, and capable'}]]

In [22]:
prompt_template= [
    {
        "role": "system",
        "content": "You are a smart AI assistant who speaks like a pirate."
    },
    {
        "role": "user",
        "content": "Where does the sun rise?"
    }
]
tokenizer.pad_token=tokenizer.eos_token

tokenized=tokenizer.apply_chat_template(
    prompt_template,
    add_generation_prompt=True,
    tokenize=False,
    padding=True,
    return_tensors="pt"
)

print(tokenized)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 29 Mar 2025

You are a smart AI assistant who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

Where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


